<a href="https://colab.research.google.com/github/raffieeey/MasterResearchAutoML/blob/master/h2oautoml/freedom/02A_H2OAutoml_Regres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing dependecies for H2O automl

In [6]:
!pip install pmlb
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip install pytictoc

In [0]:
#!pip uninstall h2o -y

## Installing H2O

In [8]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html


## Getting the number of Cores

In [9]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa itlb_multihit
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits 

##Starting H2O

In [0]:
import h2o

In [11]:
h2o.init(nthreads=4,
                 min_mem_size="2048M",
                 max_mem_size="2048M")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp_2ufxbku
  JVM stdout: /tmp/tmp_2ufxbku/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp_2ufxbku/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.3
H2O_cluster_version_age:,"14 days, 13 hours and 3 minutes"
H2O_cluster_name:,H2O_from_python_unknownUser_tjquqf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


## Creating all the metrics for Regression algorithm

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
import math
import numpy as np

class MetricsCalc:

    def __init__(self, truth, predictions):
        self.truth = truth.as_data_frame()
        self.predictions = predictions
        self._type = 'regression'

    def mae(self):
        return float(mean_absolute_error(self.truth, self.predictions))

    def mse(self):
        return float(mean_squared_error(self.truth, self.predictions))

    def msle(self):
      try:
        return float(mean_squared_log_error(self.truth, self.predictions))
      except:
        return np.nan
    
    def rmse(self):
        return math.sqrt(self.mse())

    def rmsle(self):
        return math.sqrt(self.msle())

    def r2(self):
        return float(r2_score(self.truth, self.predictions))

## Running H2O autoML with 
- 120 seconds
- Seed 69
- 2 Cores
- 2GB memory

- Tested for metrics: 
> - Deviance
> - Mean Squared error
> - Root Mean Squared error
> - Mean Absolute error
> - Root Mean Squared Logarithmic Error

- Recorded for metrics: 
> - Mean Absolute error
> - Mean Squared error
> - Mean Squared Logarithmic Error
> - Root Mean Squared error
> - Root Mean Squared Logarithmic Error
> - r2

In [13]:
from pmlb import classification_dataset_names, regression_dataset_names,fetch_data
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
from pytictoc import TicToc
import pandas as pd
from pprint import pprint

df_result = pd.DataFrame(columns=["dataset_name", "metrics_recorded", "MAE", "MSE","MSLE", "RMSE", "RMSLE", "r2"])
preds = {}
top_algo = {}
t = TicToc()


sel_clss_dtst = ['adult','agaricus-lepiota', 'churn', 'nursery', 'satimage','texture']
sel_rgrs_dtst = ['294_satellite_image','218_house_8L', '227_cpu_small', '503_wind', '344_mv','215_2dplanes']

ix = 0

for rgrs_dtst in sel_rgrs_dtst:
  for metrics_recorded in ['RMSLE']:
        df_result.loc[ix,'dataset_name'] = rgrs_dtst
        df_result.loc[ix,'metrics_recorded'] = metrics_recorded
        print("_"*80)
        print(f"*** Dataset Name: {rgrs_dtst} ***")
        X, y = fetch_data(rgrs_dtst, return_X_y=True)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=69)
        X_train = h2o.H2OFrame(X_train)
        X_test = h2o.H2OFrame(X_test)
        y_train = h2o.H2OFrame(y_train)
        y_test = h2o.H2OFrame(y_test)
        aml = H2OAutoML(seed=69, max_runtime_secs=1800,sort_metric=metrics_recorded)
        t.tic()
        aml.train(x=X_train.columns, y=X_train.cbind(y_train)[:,-1].columns[0], training_frame=X_train.cbind(y_train))
        t.toc()    
        lb = aml.leaderboard
        top_algo[rgrs_dtst] = aml.leader.actual_params
        print("*** Top algorithm: ***", lb.head(5))
        pprint(aml.leader.actual_params)
        preds[rgrs_dtst] = aml.leader.predict(X_test).as_data_frame()
        p = MetricsCalc(y_test,preds[rgrs_dtst])
        df_result.loc[ix,'MAE'] = p.mae()
        df_result.loc[ix,'MSE'] = p.mse()
        df_result.loc[ix,'MSLE'] = p.msle()
        df_result.loc[ix,'RMSE'] = p.rmse()
        df_result.loc[ix,'RMSLE'] = p.rmsle()
        df_result.loc[ix,'r2'] = p.r2()
        ix+=1

________________________________________________________________________________
*** Dataset Name: 294_satellite_image ***
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Elapsed time is 1676.250712 seconds.
*** Top algorithm: *** 

model_id,rmsle,mean_residual_deviance,rmse,mse,mae
StackedEnsemble_AllModels_AutoML_20200528_104839,0.136229,0.424331,0.651407,0.424331,0.344798
StackedEnsemble_BestOfFamily_AutoML_20200528_104839,0.139663,0.441179,0.664213,0.441179,0.350759
GBM_grid__1_AutoML_20200528_104839_model_24,0.143161,0.457669,0.676512,0.457669,0.352302
GBM_4_AutoML_20200528_104839,0.144209,0.461972,0.679685,0.461972,0.356294
GBM_grid__1_AutoML_20200528_104839_model_19,0.144997,0.465898,0.682567,0.465898,0.37095



{'base_models': [{'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'GBM_grid__1_AutoML_20200528_104839_model_24',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'GBM_4_AutoML_20200528_104839',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'GBM_grid__1_AutoML_20200528_104839_model_19',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
         

model_id,rmsle,mean_residual_deviance,rmse,mse,mae
StackedEnsemble_AllModels_AutoML_20200528_111647,0.616349,8.59869e+08,29323.5,8.59869e+08,15340.9
StackedEnsemble_BestOfFamily_AutoML_20200528_111647,0.620987,8.67327e+08,29450.4,8.67327e+08,15391.3
GBM_grid__1_AutoML_20200528_111647_model_5,0.62329,8.84438e+08,29739.5,8.84438e+08,15628.1
XGBoost_grid__1_AutoML_20200528_111647_model_1,0.624816,9.74916e+08,31223.6,9.74916e+08,16673.1
GBM_grid__1_AutoML_20200528_111647_model_3,0.630845,9.70012e+08,31145,9.70012e+08,16165.1



{'base_models': [{'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'GBM_grid__1_AutoML_20200528_111647_model_5',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'XGBoost_grid__1_AutoML_20200528_111647_model_1',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'GBM_grid__1_AutoML_20200528_111647_model_3',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'K

...


{'base_models': [{'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'DeepLearning_grid__1_AutoML_20200528_121343_model_1',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'GBM_5_AutoML_20200528_121343',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'GBM_grid__1_AutoML_20200528_121343_model_5',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
  

model_id,rmsle,mean_residual_deviance,rmse,mse,mae
XGBoost_1_AutoML_20200528_124034,nan,0.201262,0.448622,0.201262,0.302845
DeepLearning_grid__1_AutoML_20200528_124034_model_1,nan,0.0729153,0.270028,0.0729153,0.193439
StackedEnsemble_BestOfFamily_AutoML_20200528_124034,nan,0.0521621,0.22839,0.0521621,0.166724
GBM_grid__1_AutoML_20200528_124034_model_1,nan,0.075402,0.274594,0.075402,0.191866
GBM_1_AutoML_20200528_124034,nan,0.0155679,0.124771,0.0155679,0.0870356



{'backend': 'auto',
 'booster': 'gbtree',
 'build_tree_one_node': False,
 'calibrate_model': False,
 'calibration_frame': None,
 'categorical_encoding': 'AUTO',
 'checkpoint': None,
 'col_sample_rate': 0.8,
 'col_sample_rate_per_tree': 0.8,
 'colsample_bylevel': 0.8,
 'colsample_bytree': 0.8,
 'distribution': 'AUTO',
 'dmatrix_type': 'auto',
 'eta': 0.3,
 'export_checkpoints_dir': None,
 'fold_assignment': 'Modulo',
 'fold_column': None,
 'gamma': 0.0,
 'gpu_id': 0,
 'grow_policy': 'depthwise',
 'ignore_const_cols': True,
 'ignored_columns': [],
 'keep_cross_validation_fold_assignment': False,
 'keep_cross_validation_models': False,
 'keep_cross_validation_predictions': True,
 'learn_rate': 0.3,
 'max_abs_leafnode_pred': 0.0,
 'max_bins': 256,
 'max_delta_step': 0.0,
 'max_depth': 10,
 'max_leaves': 0,
 'max_runtime_secs': 0.0,
 'min_child_weight': 5.0,
 'min_data_in_leaf': 0.0,
 'min_rows': 5.0,
 'min_split_improvement': 0.0,
 'min_sum_hessian_in_leaf': 100.0,
 'model_id': 'XGBoost_1

model_id,rmsle,mean_residual_deviance,rmse,mse,mae
DeepLearning_grid__2_AutoML_20200528_130929_model_2,nan,4.40865,2.09968,4.40865,1.68956
DeepLearning_grid__3_AutoML_20200528_130929_model_1,nan,1.39052,1.1792,1.39052,0.941155
XGBoost_grid__1_AutoML_20200528_130929_model_1,nan,1.24074,1.11389,1.24074,0.889114
GBM_grid__1_AutoML_20200528_130929_model_2,nan,1.26327,1.12395,1.26327,0.896529
DeepLearning_grid__2_AutoML_20200528_130929_model_1,nan,1.21338,1.10153,1.21338,0.879383



{'activation': 'RectifierWithDropout',
 'adaptive_rate': True,
 'autoencoder': False,
 'average_activation': 0.0,
 'balance_classes': False,
 'categorical_encoding': 'AUTO',
 'checkpoint': None,
 'class_sampling_factors': None,
 'classification_stop': 0.0,
 'col_major': False,
 'diagnostics': True,
 'distribution': 'AUTO',
 'elastic_averaging': False,
 'elastic_averaging_moving_rate': 0.9,
 'elastic_averaging_regularization': 0.001,
 'epochs': 0.17478400645252365,
 'epsilon': 1e-08,
 'export_checkpoints_dir': None,
 'export_weights_and_biases': False,
 'fast_mode': True,
 'fold_assignment': 'Modulo',
 'fold_column': None,
 'force_load_balance': True,
 'hidden': [200, 200],
 'hidden_dropout_ratios': [0.2, 0.2],
 'huber_alpha': 0.9,
 'ignore_const_cols': True,
 'ignored_columns': [],
 'initial_biases': None,
 'initial_weight_distribution': 'UniformAdaptive',
 'initial_weight_scale': 1.0,
 'initial_weights': None,
 'input_dropout_ratio': 0.2,
 'keep_cross_validation_fold_assignment': Fal

## Looking at the predictions

In [14]:
preds

{'215_2dplanes':        predict
 0    -0.450173
 1    -1.746455
 2    -6.551156
 3    -2.815085
 4    -4.000388
 ...        ...
 4072  2.387203
 4073  1.507593
 4074  1.247749
 4075  6.340747
 4076  1.134694
 
 [4077 rows x 1 columns], '218_house_8L':            predict
 0     52253.277572
 1     26789.344483
 2     18053.676144
 3     33027.180055
 4     20331.519040
 ...            ...
 2274  42072.364108
 2275  60967.268341
 2276  17794.309967
 2277  32483.956416
 2278  35516.680137
 
 [2279 rows x 1 columns], '227_cpu_small':        predict
 0    94.488889
 1    89.519735
 2    86.361177
 3    93.990541
 4    90.877583
 ..         ...
 815  89.089947
 816  93.800000
 817  88.488889
 818  95.407066
 819  87.112963
 
 [820 rows x 1 columns], '294_satellite_image':       predict
 0    7.075578
 1    0.951524
 2    0.889130
 3    6.070850
 4    5.099851
 ..        ...
 639  0.975257
 640  0.864112
 641  1.810953
 642  7.023342
 643  5.719328
 
 [644 rows x 1 columns], '344_mv':        

## Saving the predictions

In [0]:
for i, val in enumerate(preds):
  preds[val].to_csv(f"{val}.csv")

## Looking at the AUC

In [16]:
df_result

,dataset_name,metrics_recorded,MAE,MSE,MSLE,RMSE,RMSLE,r2
0,294_satellite_image,RMSLE,0.346274,0.458281,0.0190428,0.676965,0.137996,0.905277
1,218_house_8L,RMSLE,15164.7,8.00607e+08,0.35431,28295,0.595239,0.690671
2,227_cpu_small,RMSLE,1.98795,8.03832,0.00152963,2.83519,0.0391105,0.974661
3,503_wind,RMSLE,2.21786,8.237,0.0454116,2.87002,0.2131,0.813781
4,344_mv,RMSLE,0.280656,0.19958,NaN,0.446744,NaN,0.998136
5,215_2dplanes,RMSLE,1.21295,2.31714,NaN,1.52222,NaN,0.878711


## Looking at the mean per class error

## Looking at the top algorithm used by the AutoML

In [17]:
top_algo

{'215_2dplanes': {'activation': 'RectifierWithDropout',
  'adaptive_rate': True,
  'autoencoder': False,
  'average_activation': 0.0,
  'balance_classes': False,
  'categorical_encoding': 'AUTO',
  'checkpoint': None,
  'class_sampling_factors': None,
  'classification_stop': 0.0,
  'col_major': False,
  'diagnostics': True,
  'distribution': 'AUTO',
  'elastic_averaging': False,
  'elastic_averaging_moving_rate': 0.9,
  'elastic_averaging_regularization': 0.001,
  'epochs': 0.17478400645252365,
  'epsilon': 1e-08,
  'export_checkpoints_dir': None,
  'export_weights_and_biases': False,
  'fast_mode': True,
  'fold_assignment': 'Modulo',
  'fold_column': None,
  'force_load_balance': True,
  'hidden': [200, 200],
  'hidden_dropout_ratios': [0.2, 0.2],
  'huber_alpha': 0.9,
  'ignore_const_cols': True,
  'ignored_columns': [],
  'initial_biases': None,
  'initial_weight_distribution': 'UniformAdaptive',
  'initial_weight_scale': 1.0,
  'initial_weights': None,
  'input_dropout_ratio': 0.

In [0]:
from google.colab import files
#df_result.to_csv("data_3.csv")
#files.download('data_3.csv')



with open('data_3.txt', 'w') as f:
    f.write(str(top_algo))

#files.download('data_3.txt')